In [5]:
import sqlite3
import pandas as pd
import torch
from transformers import pipeline, AutoTokenizer
from tqdm import tqdm

In [6]:
# Connect to your SQLite database and load data
conn = sqlite3.connect("Auto_Reply_v3.db")
queries = [
    'SELECT "translation" AS Review, "Developer Reply Text" AS Reply FROM All_Reviews',
    'SELECT "translation" AS Review, "Reply" AS Reply FROM "Auto Reply Hadith"',
    'SELECT "translation" AS Review, "Reply" AS Reply FROM Auto_Reply_Reviews',
    'SELECT "translation" AS Review, "Reply" AS Reply FROM Auto_Reply_Reviews_Al_Quran'
]
dfs = [pd.read_sql_query(q, conn) for q in queries]
df = pd.concat(dfs, ignore_index=True)
conn.close()

df

,Review,Reply
0,"Alhamdulillah.. the best al quran application,...",None
1,the best quraan app you can find. it literally...,None
2,Very helpful,None
3,The recent edition does explain the source of ...,BarakAllahu feekum for your kind words and sup...
4,May Allah reward you richly rewarded,None
...,...,...
70455,\tVery good app,BarakAllahu feekum for your kind words and sup...
70456,"\tThank God, this application is really good!!...",BarakAllahu feekum for your kind words and sup...
70457,"\tMashallah, may God bless you",BarakAllahu feekum for your kind words and sup...
70458,\tThank God this application helps👍,BarakAllahu feekum for your kind words and sup...


In [7]:
# Check CUDA availability and select device
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")


CUDA is available. Using GPU: NVIDIA GeForce RTX 4070 Ti SUPER


In [8]:
# Model and tokenizer for 3-label sentiment
model_name = 'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_analyzer = pipeline(
    'sentiment-analysis',
    model=model_name,
    tokenizer=tokenizer,
    device=0 if device == "cuda" else -1,
)
print(f"Sentiment pipeline will run on: {device}")

# Map model labels to human-readable
label_map = {'LABEL_0': 'negative', 'LABEL_1': 'neutral', 'LABEL_2': 'positive'}

# Sentiment analysis with tokenizer-based truncation

def analyze_sentiment(text):
    if not isinstance(text, str) or not text.strip():
        return None
    result = sentiment_analyzer(
        text,
        truncation=True,
        max_length=512
    )
    label = result[0]['label']
    return label_map.get(label, label)

# Apply sentiment and save to CSV
df['Sentiment'] = df['Review'].apply(analyze_sentiment)
# df.to_csv('2.roberta.csv', index=False)
# print(f"Processed {len(df)} rows. Results saved to 2.roberta.csv.")


Device set to use cuda:0


Sentiment pipeline will run on: cuda


In [9]:
df

,Review,Reply,Sentiment
0,"Alhamdulillah.. the best al quran application,...",None,positive
1,the best quraan app you can find. it literally...,None,positive
2,Very helpful,None,positive
3,The recent edition does explain the source of ...,BarakAllahu feekum for your kind words and sup...,neutral
4,May Allah reward you richly rewarded,None,positive
...,...,...,...
70455,\tVery good app,BarakAllahu feekum for your kind words and sup...,positive
70456,"\tThank God, this application is really good!!...",BarakAllahu feekum for your kind words and sup...,positive
70457,"\tMashallah, may God bless you",BarakAllahu feekum for your kind words and sup...,positive
70458,\tThank God this application helps👍,BarakAllahu feekum for your kind words and sup...,positive


In [11]:
# 5. Generate automated replies based on sentiment

def generate_reply(sentiment):
    if sentiment == "positive":
        return (
            "BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!"
        )
    elif sentiment == "negative":
        return (
            "Assalamu 'alaykum dear user. We would love to make your experience better in sha Allah. If you are still facing the same issues or have any feature requests, please provide your feedback on feedback.gtaf.org. JazakAllahu khayran."
        )
    else :
        return "JazakAllahu khayran for your review! If you face any issues or have feature requests, please provide your feedback on feedback.gtaf.org."

# Apply reply generation
df['Reply'] = df['Sentiment'].apply(generate_reply)

# 6. Save the results to CSV including Review, sentiment, and reply
df.to_csv('2.roberta.csv', index=False)
print(f"Processed {len(df)} reviews. Results saved to '2.roberta.csv'.")

Processed 70460 reviews. Results saved to '2.roberta.csv'.
